In [1]:

#!pip install d2l==0.17.0
import numpy as np
import nltk
from tensorflow.keras.optimizers import RMSprop
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
import matplotlib.pyplot as plt
import pickle
import heapq

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Corpora.txt to Corpora.txt


In [3]:
f= open("Corpora.txt", "r")
data = f.read()
print(len(data))

590983


In [4]:
vocabulary = sorted(list(set(data)))

char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))
 
print(vocabulary)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '°', '¶', 'â', 'æ', 'è', 'é', 'ê', 'ï', 'ô', 'õ', 'ö', '—', '‘', '’', '“', '”', '†', '‡', '′', '″', '\ufeff']


In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
sentences = nltk.sent_tokenize(data)
print("The number of sentences is",len(sentences))

The number of sentences is 3647


In [7]:
words = nltk.word_tokenize(data)
print("number of words are", len(words))


number of words are 109113


In [8]:
unique_tokens = set(words)
print("the number of unique tokens are", len(unique_tokens))

the number of unique tokens are 11671


Data pre-processing

In [9]:
max_length = 100
steps = 5
sentences = []
next_chars = []
for i in range(0, len(data) - max_length, steps):
    sentences.append(data[i: i + max_length])
    next_chars.append(data[i + max_length])

encoding characters into boolean vectors

In [10]:
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype = np.bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

building the model

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape =(max_length, len(vocabulary))))
model.add(Dense(len(vocabulary)))
model.add(Activation('softmax'))
optimizer = RMSprop(learning_rate = 0.01)
model.compile(loss ='categorical_crossentropy', optimizer = optimizer)

In [12]:
history= model.fit(X, y, batch_size = 128, epochs = 10, validation_split=0.05, shuffle=True).history

Epoch 1/10
878/878 [==============================] - 28s 25ms/step - loss: 2.0690 - val_loss: 2.4048
Epoch 2/10
878/878 [==============================] - 21s 24ms/step - loss: 1.6691 - val_loss: 2.2890
Epoch 3/10
878/878 [==============================] - 21s 23ms/step - loss: 1.5323 - val_loss: 2.4015
Epoch 4/10
878/878 [==============================] - 21s 24ms/step - loss: 1.4531 - val_loss: 2.4618
Epoch 5/10
878/878 [==============================] - 21s 24ms/step - loss: 1.4064 - val_loss: 2.3819
Epoch 6/10
878/878 [==============================] - 21s 24ms/step - loss: 1.3634 - val_loss: 2.4238
Epoch 7/10
878/878 [==============================] - 21s 23ms/step - loss: 1.3327 - val_loss: 2.4314
Epoch 8/10
878/878 [==============================] - 21s 24ms/step - loss: 1.3484 - val_loss: 2.4381
Epoch 9/10
878/878 [==============================] - 21s 24ms/step - loss: 1.3135 - val_loss: 2.5427
Epoch 10/10
878/878 [==============================] - 21s 24ms/step - loss: 1.280

In [13]:
model.save('predictor.h5')
